In [ ]:
#%pip install -U transformers accelerate torch
#!pip install numpy==1.24.4

In [1]:
# Import library
from langchain.document_loaders import PyPDFLoader

# Create a document loader for rag_vs_fine_tuning.pdf
loader = PyPDFLoader('https://greencastlein.api.civicclerk.com/v1/Meetings/GetMeetingFileStream(fileId=90,plainText=false)')

# Load the document
data = loader.load()
print(data[0])

page_content='City of Greencastle 
Common Council 
Regular Session 
Mikayla Johnson - Clerk Treasurer 
 
 
 
April 10, 2025 | 7:00 PM 
City Hall 
One North Locust Street, P.O. Box 607 
Greencastle, Indiana 46135 
 
 
 
 
 
AGENDA 
 
I. Call to Order; Roll Call 
II. Pledge of Allegiance 
III. Public Petitions and Comments 
IV. Special Requests 
  A. Noise Ordinance Waiver Request - DePauw Alumni Reunion Weekend 2025 
  B. Noise Ordinance Waiver Request- Phi Kappa Psi Fraternity Social Event With Guests 
  C. Noise Ordinance Waiver Request- Phi Kappa Psi Fraternity Social Event With Guests 
  D. Special Event Street Closure Request- DePauw University Commencement 
V. Department Reports 
  A. Cemetery - Jason Keeney 
  B. Fire Department - Rob Frank 
  i. Fire Inspections - John Burgess 
  C. Planner - Blaine Rout 
  D. Police Department - Chris Jones 
  E. Department of Public Works - Andrew Rogers 
  i. March 2025 Summary Report 
  F. Water & Wastewater Department - Oscar King Jr. 
  i.

In [2]:
from langchain_huggingface import HuggingFacePipeline
from transformers import AutoTokenizer, pipeline, AutoModelForCausalLM
from huggingface_hub import login
from langchain_core.prompts import PromptTemplate
model_id = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
template = """
You are an assistant that summarizes city council agenda packets so that citizens know what's happening.

The document represents the pages of the packet. The first page is the overall agenda and the pages that follow correspond to items listed on page 1

For each of the agenda items, summarize the key points.
Be concise and neutral.
Return your response as a bullet-point list using plain language.
Provide a headline for each agenda item on the list.

Document:
{agenda}

Summary:
"""

llm = HuggingFacePipeline.from_model_id(model_id = model_id,
                                       task="text-generation",
                                       pipeline_kwargs={"max_new_tokens": 300})

# Invoke the model
prompt_template = PromptTemplate.from_template(template)
formatted_prompt = prompt_template.format(agenda=data)
response = llm.invoke(formatted_prompt)
print(response)

Device set to use cpu
Token indices sequence length is longer than the specified maximum sequence length for this model (22475 > 16384). Running this sequence through the model will result in indexing errors


KeyboardInterrupt: 